In [1]:
from main import grigora2, Worker, Group, expert_parallel_group_objective_function, all_reduce_function, gamma_function, Expert
import numpy as np
import math
from lazy_streams import stream
from sortedcontainers import SortedSet

## Evaluation of Lysi (renamed from Grigora)
Let's perform realistic experiments using the sharding specification generated by Lysi.

Specifically, we will use this [script](https://github.com/osayamenja/Megatron-DeepSpeed/blob/main/examples_deepspeed/MoE/ds_pretrain_gpt_350M_MoE128.sh) and train GPT-3 16x350M on four Perlmutter [GPU nodes](https://docs.nersc.gov/systems/perlmutter/architecture/#gpu-nodes) 

Below, we define the number of workers and number of GPUs per node

In [2]:
dim = 16
intra_node_width = 4.0

Next, we build the adjacency matrix. We manually obtained the alpha and beta values below via NCCL-tests mirco-benchmarks. 
We anticipate automating this network profiling procedure.

In [3]:
adjacency = np.zeros((dim, dim, 2))
intra_node_cost = (0.009, 0.014)  # (ms, ms/MB)
inter_node_cost = (0.03, 0.054)

Note that each GPU connects to a separate NIC in the Perlmutter; thus, there are only two types of links: intra-node NVLink and internode NIC connections.

In [4]:
for ii in range(adjacency.shape[0]):
    for jj in range(adjacency.shape[0]):
        if ii != jj and math.floor(jj / intra_node_width) == math.floor(ii / intra_node_width):
            # intra-node
            adjacency[ii, jj] = intra_node_cost
        else:
            # inter-node
            adjacency[ii, jj] = inter_node_cost

Below outlines the theoretical FLOPS of the tensor core in the A100 GPU, which comprises our testbed. We used the values from official [documentation](https://images.nvidia.com/aem-dam/en-zz/Solutions/data-center/nvidia-ampere-architecture-whitepaper.pdf), *but* we obtained the realistic scaling factor from empirical measurements. 

Note this scale is much less than the 75% reported by [NVIDIA](https://forums.developer.nvidia.com/t/about-gpu-peak-performance/264462/5) but aligns with the [literature](https://ieeexplore.ieee.org/document/9415606), which details 40% utilization for $4096\times4096$ matrices on the V100.

The expert matrix GEMMs for GPT-3 MoE are described below. Note that $\bigotimes$ denotes matrix multiplication, $s$ sequence length, $h$ hidden size, and $b$ batch size.
$$(s\cdot b,\; h)\bigotimes (h, \;4h) = (2048\cdot 4, \;1024) \bigotimes (1024, \;4096)$$   

In [5]:
a100_theoretical_flop_per_ms = 312 * 1E9
realistic_scaling_factor = 0.43
real_flops = int(math.ceil(realistic_scaling_factor * a100_theoretical_flop_per_ms))

In [6]:
mem = 32
w = []
for ii in range(adjacency.shape[0]):
    w.append(Worker(ii, real_flops, mem))

We define the experts below.

In [7]:
n_exp = 64
exp = []
exp_flops = 16 * 4 * 2048 * (1024 ** 2)
for ii in range(n_exp):
    exp.append(exp_flops)

Ensure to check this [file](grigora_manuscript.pdf) for more details.

In [8]:
p2p_buf_mb = 16
p2p_fr = 4
all_r_buf = 512

gamma_arguments = {Group.NUM_LAYERS: 24,
                   Group.GLOBAL_BATCH_SIZE: 256,
                   Group.MINI_BATCH_SIZE: 4,
                   Group.MOE_FREQUENCY: 2,
                   Group.RECOMPUTATION_AMOUNT: 1}

In [9]:
shard_spec, inv = grigora2(a=adjacency,
                               obj=expert_parallel_group_objective_function,
                               all_reduce_func=all_reduce_function,
                               gamma=gamma_function,
                               p2p_buffer_size=p2p_buf_mb,
                               p2p_freq=p2p_fr,
                               all_reduce_buffer_size=all_r_buf,
                               workers=w,
                               expert_workload=exp,
                               gamma_args=gamma_arguments)
print(shard_spec.subsets())

[{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}]


As shown above, Lysi produces a sharding specification where ranks of workers are grouped into communication-optimal groups.

In [7]:
mem_capacity = [2, 2, 3, 4, 5, 6, 7, 8]
stream(mem_capacity).reduce(lambda m1, m2: m1 + m2)

37

In [8]:
def distinctify(w1: Worker, w2: Worker):
    if w1 == w2:
        return w1
    return w1, w2

In [22]:
foo = [Worker(1, 0, 0), Worker(1, 0, 0), Worker(1, 0, 0), Worker(1, 0, 0), 
       Worker(3, 0, 0), Worker(1, 0, 0), Worker(1, 0, 0) , Worker(1, 8, 9)]
print(len(set(foo)))

2


In [24]:
streamm = stream(foo).map(lambda f: f.worker_id)
print(streamm.reduce(lambda m1, m2: m1 + m2))
print(streamm.reduce(lambda m1, m2: m1 * m2))

10
3


In [26]:
print(stream(foo).filter(lambda f: f.mem_capacity >= 2).to_string())

Worker(id=1, flops=8, mem_capacity=9)


In [2]:
def match(budget: Expert, ss: SortedSet) -> Expert:
    if budget in ss:
        ss.discard(budget)
        return budget
    floor = ss.bisect_left(budget) - 1
    ceiling = ss.bisect_right(budget)
    if floor >= 0 and ceiling <= len(ss):
        most_similar_cost = budget.cost_most_similar_to([ss[floor], ss[ceiling]])
        ss.discard(most_similar_cost)
        return most_similar_cost
    return ss[floor] if floor >= 0 else ss[ceiling]

In [9]:
sl = SortedSet([Expert(0), Expert(6), Expert(1100)])
bar = Expert(4)
match(bar, sl)

Expert(id=0, compute_cost=6)